In [1]:
import os
os.chdir("/media/pc1/Ubuntu/Extend_Data/em_Thanh/medrag_colpali")
os.environ["HF_HOME"] = "/media/pc1/Ubuntu/Extend_Data/hf_models"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
from src.rag.rag import Rag
from typing import List

rag = Rag()


/media/pc1/Ubuntu/Extend_Data/em_Thanh/medrag_colpali/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.08s/it]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda


In [3]:
def get_answer_from_medgemma(
        query: str,
        images_path: List[str],
    ) -> str:

        fused_query = (
            "You are a professional clinician specializing in tropical and infectious disease diagnosis.\n"
            "Please evaluate the patient case and reason through four stages (Information Gathering, Hypothesis Generation, "
            "Hypothesis Testing, Reflection & Final Diagnosis). Be concise and grounded.\n\n"
            "# PATIENT CASE\n"
            f"{query}\n\n"
            "# CONTEXT — RETRIEVED_DOCS\n"
            f"No Infomation\n\n"
            "# CONTEXT — KNOWLEDGE_GRAPH\n"
            f"No Information\n\n"
            "These are relative information now give yyou diagnosis for the PATIENT CASE. Please ground your reasoning in the CONTEXT sections when possible.\n"
            "Let’s think step by step." \
            "\n# YOUR DIAGNOSIS"
        )
        return rag.get_answer_from_medgemma(fused_query, images_path)

In [4]:
import json

path_file = "data/processed/93_cases"
files = os.listdir(path_file)
text_lst = []
img_lst = []

for file in files:
    file_path = os.path.join(path_file, file)
    imgs = []

    # Load JSON
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # Combine all fields except "Extracted Figure"
    combined_text = ""
    for key, value in data.items():
        if key == "query":
            combined_text += value

    # Get image links
    image_paths = []
    for figure_group in data.get("Extracted Figure", []):
        for figure in figure_group:
            path = figure.get("Link")
            path = os.path.join('data/raw', path)
            if path:
                image_paths.append(path)

    # Final combined text
    text_lst.append(combined_text)
    img_lst.append(image_paths)


In [5]:
import json
import os

def safe_append_json(filepath, record):
    # Ensure the file exists with a valid empty JSON array
    if not os.path.exists(filepath):
        with open(filepath, "w", encoding="utf-8") as f:
            json.dump([], f)

    # Load safely, reset if corrupted
    with open(filepath, "r+", encoding="utf-8") as f:
        try:
            data = json.load(f)
            if not isinstance(data, list):
                data = []  # Force reset if JSON is not a list
        except json.JSONDecodeError:
            data = []  # Reset if file is empty or invalid

        data.append(record)

        # Overwrite with updated JSON
        f.seek(0)
        json.dump(data, f, indent=4, ensure_ascii=False)
        f.truncate()

In [6]:
output_file = "data/processed/medgemma/result_gemma3.json"
processed_files = []
with open(output_file, "r", encoding="utf-8") as f:
    data = json.load(f)
for case in data:
    for key, value in case.items():
            if key in ['original_file']:
                processed_files.append(value)

In [7]:
len(processed_files)

181

In [8]:
for query, imgs, file in zip(text_lst, img_lst, files):
    if file not in processed_files:
        answer = get_answer_from_medgemma(
            query=query,
            images_path=imgs if imgs else [],
        )
        print(f"Answer: \n{answer}")

        record = {
            "query": query,
            "images_used": imgs,
            "answer": answer,
            "original_file": file
        }
        print(record)

        # Load existing data, append, and rewrite
        safe_append_json(output_file, record)
  


[{'role': 'system', 'content': [{'type': 'text', 'text': '\n            You are a clinician specialized in Tropical & Infectious Diseases.\n\n            # Objective\n            Evaluate a patient case (text + optional images) together with two context blocks:\n            (1) retrieved document excerpts and (2) knowledge-graph facts. Use this information to arrive at the most likely diagnosis with clear clinical reasoning.\n\n            # Reasoning Approach\n            • Use System 1 (intuitive, pattern recognition) and System 2 (analytic, hypothesis testing).\n            • Think step by step and structure your reasoning using these four stages:\n            1) Information Gathering — key signs/symptoms and duration; epidemiologic context (travel, residence, occupation, exposures); relevant comorbidities/medications; red-flag features (e.g., hemorrhage, shock, AMS, jaundice).\n            2) Hypothesis Generation — organize by anatomical system and by timing/progression; consider 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Answer: 
Based on the patient's presentation, history, and investigations, the most likely diagnosis is **Disseminated Histoplasmosis** complicated by **Acquired Immunodeficiency Syndrome (AIDS)**.

Here's the reasoning:

1.  **Information Gathering:**
    *   **Key Signs/Symptoms:** Fever, weight loss, cough, lymphadenopathy (chest and hilar), skin lesions (eczematoid), possible thrush, anemia, leukopenia, and a history of malaria.
    *   **Duration:** Symptoms have been present for 5 weeks, with a recent decline in health.
    *   **Epidemiologic Context:** Native of Honduras, lived on a farm, exposure to cattle, immigration to Boston.
    *   **Comorbidities/Medications:** History of malaria treated with unknown medication, azithromycin, and recent use of cefuroxime, doxycycline, and clarithromycin.
    *   **Red-Flag Features:** Weight loss, lymphadenopathy, possible thrush, and the patient's cachectic appearance.

2.  **Hypothesis Generation:**
    *   **Systemic:** Disseminated 